In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_theme(style="whitegrid")

In [ ]:
order_item = pd.read_csv("../../../downloads/olist_redefined/order_item.tsv", sep='\t').drop_duplicates()
order_item

In [ ]:
# order_product = order_item[['order_id','product_id', 'seller_id', 'price', 'freight_value']]
order_product = order_item.drop(columns=['shipping_limit_date'])
order_product

In [ ]:
order_product.drop_duplicates().shape

In [ ]:
order_product['product_id'].nunique()

In [ ]:
product = pd.read_csv("../../../downloads/olist_redefined/product.tsv", sep='\t').drop_duplicates()
product

In [ ]:
product['product_id'].nunique()

In [ ]:
order_product_info = order_product.merge(product, on='product_id', how='inner').drop_duplicates()
order_product_info

In [ ]:
order_product_info.isna().value_counts()

In [ ]:
null_product_category = order_product_info[order_product_info['category'].isna()]
null_product_category_seller = null_product_category['seller_id'].unique()

In [ ]:
order_info_of_null_product_category_seller = order_product_info[order_product_info['seller_id'].isin(null_product_category_seller)]
order_info_of_null_product_category_seller

In [ ]:
totl_selling_product_count_by_seller = order_info_of_null_product_category_seller.groupby('seller_id').agg({'product_id': 'count'})
totl_selling_product_count_by_seller.rename(columns={'product_id': "total_selling_produt_count"}, inplace=True)
totl_selling_product_count_by_seller.sort_values('total_selling_produt_count', ascending=False)

In [ ]:
totl_selling_product_count_by_seller[totl_selling_product_count_by_seller['total_selling_produt_count'] > 1].value_counts()

In [ ]:
tmp = order_info_of_null_product_category_seller.groupby('seller_id').agg({'category': 'nunique'}).sort_values('category')
tmp[tmp['category']==1]

In [ ]:
tmp = order_product_info[order_product_info.seller_id == "4e922959ae960d389249c378d1c939f5"]
tmp['category'].unique()

판매자의 물품의 카테고리가 정의되어 있지 않은 경우에, 그 판매자가 판매 중인 모든 제품의 카테고리와 크기, 무게를 비교하여 넣으려고 했으나, 완벽한(100%) 추정이 아니므로 제외 결정

In [ ]:
clean_order_product_info = order_product_info.dropna()
clean_order_product_info

In [ ]:
# 판매 가격 변동이 있는 제품. 원인은 모름
tmp = clean_order_product_info.groupby('product_id', as_index=False).agg(price_count=('price', 'nunique'))
price_changed = tmp[tmp['price_count'] > 1]
price_changed.shape


In [ ]:
price_changed['price_count'].value_counts()

In [ ]:
# 동일 상품을 여러 판매자가 판매하는 경우도 있음
max_price_changed_product_id = price_changed[price_changed['price_count'] == 27]['product_id'].values[0]
order_product_info[order_product_info['product_id'] == max_price_changed_product_id]['seller_id'].unique()

In [ ]:
price_dist = clean_order_product_info['price'].value_counts()
price_dist

# sales

In [79]:
clean_orders = clean_order_product_info.drop(columns=['weight_g','length_cm','height_cm','width_cm'])
clean_orders['selling_price'] = clean_orders['price'] + clean_orders['freight_value']
clean_orders

In [80]:
order_status = pd.read_csv("../../../downloads/olist_redefined/order_status.tsv", sep='\t').drop_duplicates()
complete_order = order_status[order_status['status'] == 'delivered_customer']
complete_order

In [81]:
payment = pd.read_csv("../../../downloads/olist_redefined/payment.tsv", sep='\t').drop_duplicates()
complete_payment = payment[payment['order_id'].isin(complete_order['order_id'])]
complete_payment = complete_payment.sort_values(['order_id', 'payment_sequential']).drop_duplicates()
complete_payment

In [82]:
complete_revenue = complete_payment.groupby('order_id', as_index=False).agg(revenue=('payment_value', 'sum'))
complete_revenue

In [83]:
complete_sales = clean_orders[clean_orders['order_id'].isin(complete_revenue['order_id'])]
complete_sales.sort_values(['order_id', 'product_id'])

In [84]:
total_sales_by_category = complete_sales.groupby('category', as_index=False).agg(total_selling_price=('selling_price', 'sum'))
total_sales_by_category = total_sales_by_category.sort_values('total_selling_price', ascending=False)
total_sales_by_category['proportion'] = total_sales_by_category['total_selling_price'] / total_sales_by_category['total_selling_price'].sum() * 100

In [85]:
# health_beauty 카테고리에서 제품별 비중
health_beauty_order = complete_sales[complete_sales['category'] == 'health_beauty']
health_beauty_order

In [86]:
popular_product = health_beauty_order.groupby('product_id', as_index=False).agg(total_selling_price=('selling_price', 'sum'), order_count=('order_id', 'count')).sort_values('total_selling_price', ascending=False)
popular_product['avg_price'] = popular_product['total_selling_price'] / popular_product['order_count']
popular_product['total_selling_price_cumsum'] = popular_product['total_selling_price'].cumsum()
popular_product['total_selling_price_proportion'] = popular_product['total_selling_price'] / popular_product['total_selling_price'].sum()
popular_product['total_selling_price_proportion_cumsum'] = popular_product['total_selling_price_proportion'].cumsum()
popular_product

In [88]:
popular_product[popular_product['product_id'] == '154e7e31ebfa092203795c972e5804a6']

In [82]:
popular_product_sorted = popular_product.sort_values('total_selling_price', ascending=False).reset_index(drop=True)
total_products = len(popular_product_sorted)
popular_product_sorted['product_rank'] = popular_product_sorted.index + 1
popular_product_sorted['rank_proportion'] = popular_product_sorted['product_rank'] / total_products
popular_product_sorted

In [83]:
top_product = popular_product_sorted[popular_product_sorted['total_selling_price_proportion_cumsum'] <= 0.8]
top_product
# top_product.to_csv('top80.csv',index=False)

In [84]:
top_product['total_selling_price'].describe()

In [103]:
popular_product_sorted['order_count'].describe()

In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 가정: 'popular_product_sorted' 변수에 전체 2397건의 데이터가 로드되어 있다고 가정합니다.
# 이 데이터프레임에는 'order_count', 'avg_price', 'total_selling_price' 열이 있어야 합니다.

# 1. 그룹 분류 기준점 재설정
# 주문 수 기준: 상위 25% (75th percentile)를 사용하여 '많이 팔림'을 정의
order_count_threshold = popular_product_sorted['order_count'].quantile(0.75)
# 평균 판매가 기준: 중앙값(median)을 그대로 사용
median_avg_price = popular_product_sorted['avg_price'].median()

print("--- Analysis Thresholds (Full Dataset, Revised) ---")
print(f"Order Count Threshold (Top 25%): {order_count_threshold:.2f}")
print(f"Median Average Price: {median_avg_price:.2f}\n")


# 2. 제품을 4개 그룹으로 분류하는 함수 정의 (기준점 변수명 변경)
def classify_product_english(row):
    is_high_volume = row['order_count'] >= order_count_threshold # 변경된 기준점 사용
    is_high_price = row['avg_price'] >= median_avg_price
    
    if is_high_volume and is_high_price:
        return 'Star Products'
    elif is_high_volume and not is_high_price:
        return 'Volume Drivers'
    elif not is_high_volume and is_high_price:
        return 'Niche Gems'
    else:
        return 'Question Marks'

# 3. 함수를 적용하여 'group' 열 생성
popular_product_sorted['group'] = popular_product_sorted.apply(classify_product_english, axis=1)

# --- 명확성을 위해 그룹 정의를 정의하고 표시 ---
group_definitions = {
    'Star Products': '주문량과 판매가 모두 높은 제품. 매출의 핵심 동력.',
    'Volume Drivers': '주문량은 많지만 판매가는 낮은 제품. 폭넓은 고객층을 유인.',
    'Niche Gems': '주문량은 적지만 판매가는 높은 제품. 특정 고가치 시장을 공략.',
    'Question Marks': '주문량과 판매가 모두 낮은 제품. 전체 매출에 미치는 영향이 비교적 적음.'
}

print("--- Group Definitions ---")
for group, definition in group_definitions.items():
    print(f"- {group}: {definition}")
print("-" * 25)

# 4. 전체 데이터에 대한 그룹 통계 표시
print("\n--- Product Count by Group (Full Dataset) ---")
print(popular_product_sorted['group'].value_counts())


# 5. 시각화
plt.figure(figsize=(10, 10))
plt.xscale('log')
plt.yscale('log')

sns.scatterplot(
    data=popular_product_sorted,
    x='order_count',
    y='avg_price',
    hue='group',
    size='total_selling_price',
    sizes=(20, 800), # 점의 최소 크기를 더 작게 조정하여 롱테일을 표현
    alpha=0.6, # 점이 많으므로 투명도를 살짝 높임
    palette='Set1'
)

plt.gca().set_aspect('equal', adjustable='box')

# 기준선도 새로운 기준으로 업데이트
plt.axhline(median_avg_price, color='red', linestyle='--', label=f'Median Avg Price ({median_avg_price:.0f})')
plt.axvline(order_count_threshold, color='blue', linestyle=':', label=f'Top 25% Order Count ({order_count_threshold:.0f})')

plt.title('Product Segmentation by Order Count and Average Price (Full Dataset)', fontsize=16)
plt.xlabel('Order Count (Log Scale)', fontsize=12)
plt.ylabel('Average Selling Price (Log Scale)', fontsize=12)
plt.legend(title='Product Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()

plt.savefig('full_data_segmentation_chart.png')
print("\nAnalysis chart for the full dataset has been saved as 'full_data_segmentation_chart.png'")



In [107]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# # 가정: 'popular_product_sorted' 변수에 전체 데이터가 로드되어 있다고 가정합니다.

# # --- 1. 데이터 준비 (K-Means를 위한 스케일링) ---
# # K-Means는 값의 범위(scale)에 민감하므로, 로그 변환과 표준화를 적용합니다.
# # 로그 변환은 데이터의 왜곡을 줄여줍니다.
# features = popular_product_sorted[['order_count', 'avg_price']].copy()
# log_features = features.apply(np.log1p)

# # 표준화는 각 변수의 중요도를 동등하게 만들어줍니다.
# scaler = StandardScaler()
# scaled_features = scaler.fit_transform(log_features)


# # --- 2. 최적의 클러스터 개수(K) 찾기 (Elbow Method) ---
# inertia = []
# k_range = range(1, 11)
# for k in k_range:
#     kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
#     kmeans.fit(scaled_features)
#     inertia.append(kmeans.inertia_)

# # Elbow Method 시각화
# plt.figure(figsize=(8, 5))
# plt.plot(k_range, inertia, marker='o')
# plt.title('Elbow Method for Optimal K')
# plt.xlabel('Number of Clusters (K)')
# plt.ylabel('Inertia')
# plt.grid(True)
# plt.show()
# # 그래프에서 팔꿈치처럼 꺾이는 지점이 최적의 K입니다. (보통 3, 4, 또는 5)


# # --- 3. K-Means 실행 및 클러스터 할당 ---
# # 위 그래프를 보고 최적의 K를 결정합니다. 여기서는 4로 가정하겠습니다.
# optimal_k = 4
# kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
# popular_product_sorted['cluster'] = kmeans.fit_predict(scaled_features)


# # --- 4. 결과 시각화 및 해석 ---
# print(f"\n--- Analysis Results with K={optimal_k} ---")

# # 각 클러스터의 통계적 특성 분석
# cluster_summary = popular_product_sorted.groupby('cluster')[['order_count', 'avg_price', 'total_selling_price']].mean().sort_values('total_selling_price', ascending=False)
# print("\n--- Mean Characteristics of Each Cluster ---")
# print(cluster_summary)

# # 클러스터링 결과 시각화
# plt.figure(figsize=(12, 10))
# plt.xscale('log')
# plt.yscale('log')

# sns.scatterplot(
#     data=popular_product_sorted,
#     x='order_count',
#     y='avg_price',
#     hue='cluster',  # 새로운 클러스터 기준으로 색상 구분
#     size='total_selling_price',
#     sizes=(20, 1000),
#     alpha=0.7,
#     palette='Set1' # 대비가 높은 색상 팔레트
# )

# plt.gca().set_aspect('equal', adjustable='box')

# plt.title(f'K-Means Clustering Results (K={optimal_k})', fontsize=16)
# plt.xlabel('Order Count (Log Scale)', fontsize=12)
# plt.ylabel('Average Selling Price (Log Scale)', fontsize=12)
# plt.legend(title='Cluster ID', bbox_to_anchor=(1.05, 1), loc='upper left')
# plt.grid(True)
# plt.tight_layout()

# plt.savefig('kmeans_clustering_result.png')
# print("\nK-Means clustering analysis chart saved as 'kmeans_clustering_result.png'")
